# <center>TW50 LSTM Forecasting</center>

> Authors:
> - D11202805 - Ian Joseph Chandra
> - M11002818 - Wilfrid Azariah

## Step 1: Import Libraries and Load Data

In [11]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras import Sequential
from tensorflow.keras.layers import LSTM, Dropout, Dense
from datetime import timedelta

# Step 1: Load and Prepare the Data
file_path = "../FTSE TWSE Taiwan 50 Index.csv"
df = pd.read_csv(file_path, usecols=['Date', 'Price Index'])  # Load the data

df['Date'] = pd.to_datetime(df['Date'])  # Convert the 'Date' column to datetime format
df.set_index('Date', inplace=True)
df = df.iloc[::-1]  # Reverse the DataFrame to have the oldest data first

scaler = MinMaxScaler()
df['Price Index'] = scaler.fit_transform(df[['Price Index']])

# Remove rows with NaN values
df = df.dropna()

df

,Price Index
Date,
2019-05-10,0.131186
2019-05-13,0.117371
2019-05-14,0.111688
2019-05-15,0.112493
2019-05-16,0.105687
...,...
2024-05-03,0.935136
2024-05-06,0.955959
2024-05-07,0.974304


## Step 2: Split the Data for Training and Testing

In [12]:
train_size = int(len(df) * 0.8)
train_data, test_data = df.iloc[:train_size], df.iloc[train_size:]

train_data

,Price Index
Date,
2019-05-10,0.131186
2019-05-13,0.117371
2019-05-14,0.111688
2019-05-15,0.112493
2019-05-16,0.105687
...,...
2023-05-03,0.503192
2023-05-04,0.507080
2023-05-05,0.510297


In [13]:
test_data

,Price Index
Date,
2023-05-10,0.515448
2023-05-11,0.508232
2023-05-12,0.502318
2023-05-15,0.501634
2023-05-16,0.518501
...,...
2024-05-03,0.935136
2024-05-06,0.955959
2024-05-07,0.974304


## Step 3: Prepare Training Data for LSTM

In [23]:
train_data['Price Index'].values.reshape(-1, 1).shape

(974, 1)

In [15]:
def prepare_data(data, look_back=1):
    X, y = [], []
    for i in range(len(data) - look_back):
        X.append(data[i:(i + look_back), 0])
        y.append(data[i + look_back, 0])
    return np.array(X), np.array(y)


look_back = 20  # Adjust this based on your data and model performance

train_sequence = train_data['Price Index'].values.reshape(-1, 1)
test_sequence = test_data['Price Index'].values.reshape(-1, 1)

X_train, y_train = prepare_data(train_sequence, look_back)
X_test, y_test = prepare_data(test_sequence, look_back)

X_train = np.reshape(X_train, (X_train.shape[0], X_train.shape[1], 1))
X_test = np.reshape(X_test, (X_test.shape[0], X_test.shape[1], 1))

train_sequence
# X_train.shape

array([[0.13118617],
       [0.11737123],
       [0.11168811],
       [0.11249277],
       [0.10568733],
       [0.096551  ],
       [0.09714214],
       [0.09816137],
       [0.09875672],
       [0.08406139],
       [0.08767763],
       [0.08588108],
       [0.08393937],
       [0.08248888],
       [0.08981077],
       [0.10118438],
       [0.10443353],
       [0.09667406],
       [0.09895657],
       [0.0949259 ],
       [0.11015031],
       [0.11460908],
       [0.11606378],
       [0.10910477],
       [0.10428522],
       [0.10282315],
       [0.10741446],
       [0.12637187],
       [0.12709659],
       [0.13009225],
       [0.12388425],
       [0.11752164],
       [0.11015241],
       [0.1228566 ],
       [0.11897003],
       [0.13518318],
       [0.13168369],
       [0.11897424],
       [0.12173218],
       [0.12236013],
       [0.12009446],
       [0.11576611],
       [0.12580808],
       [0.13109045],
       [0.12875956],
       [0.13412818],
       [0.13596996],
       [0.128

In [16]:
y_train.shape

(954,)

In [17]:
X_test.shape

(224, 20, 1)

In [18]:
y_test.shape

(224,)

## Step 4: Build and Train the LSTM Model


In [19]:
model = Sequential()
model.add(LSTM(units=50, return_sequences=True, input_shape=(X_train.shape[1], 1)))  # First LSTM layer
model.add(Dropout(0.2))  # Dropout layer to prevent overfitting
model.add(LSTM(units=50, return_sequences=True))  # Second LSTM layer
model.add(Dropout(0.2))  # Dropout layer to prevent overfitting
model.add(LSTM(units=50))  # Third LSTM layer
model.add(Dropout(0.2))  # Dropout layer to prevent overfitting
model.add(Dense(units=1))  # Output layer

model.compile(optimizer='adam', loss='mean_squared_error')
model.summary()


/Users/ijosh/Documents/GitHub/time-series/venv/lib/python3.11/site-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ lstm_3 (LSTM)                   │ (None, 20, 50)         │        10,400 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_3 (Dropout)             │ (None, 20, 50)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_4 (LSTM)                   │ (None, 20, 50)         │        20,200 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_4 (Dropout)             │ (None, 20, 50)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_5 (LSTM)                   │ (None, 50)             │        20,200 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_5 (Dropout)             │ (None, 50)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 1)              │            51 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 50,851 (198.64 KB)

 Trainable params: 50,851 (198.64 KB)

 Non-trainable params: 0 (0.00 B)

In [20]:
model.fit(X_train, y_train, epochs=100, batch_size=32)

Epoch 1/100
30/30 ━━━━━━━━━━━━━━━━━━━━ 6s 20ms/step - loss: 0.1116
Epoch 2/100
30/30 ━━━━━━━━━━━━━━━━━━━━ 1s 18ms/step - loss: 0.0072
Epoch 3/100
30/30 ━━━━━━━━━━━━━━━━━━━━ 1s 17ms/step - loss: 0.0050
Epoch 4/100
30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step - loss: 0.0053
Epoch 5/100
30/30 ━━━━━━━━━━━━━━━━━━━━ 1s 20ms/step - loss: 0.0045
Epoch 6/100
30/30 ━━━━━━━━━━━━━━━━━━━━ 1s 19ms/step - loss: 0.0035
Epoch 7/100
30/30 ━━━━━━━━━━━━━━━━━━━━ 1s 39ms/step - loss: 0.0038
Epoch 8/100
30/30 ━━━━━━━━━━━━━━━━━━━━ 1s 22ms/step - loss: 0.0038
Epoch 9/100
30/30 ━━━━━━━━━━━━━━━━━━━━ 1s 19ms/step - loss: 0.0042
Epoch 10/100
30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step - loss: 0.0041
Epoch 11/100
30/30 ━━━━━━━━━━━━━━━━━━━━ 1s 21ms/step - loss: 0.0034
Epoch 12/100
30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step - loss: 0.0030
Epoch 13/100
30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step - loss: 0.0038
Epoch 14/100
30/30 ━━━━━━━━━━━━━━━━━━━━ 1s 22ms/step - loss: 0.0036
Epoch 15/100
30/30 ━━━━━━━━━━━━━━━━━━━━ 1s 28ms/step - lo

## Step 5: Forecasting the Next 5 Business Days of Price Index

In [25]:
last_sequence = df['Price Index'][-look_back:].values.reshape(-1, 1)
last_sequence

array([[0.99665303],
       [0.98910078],
       [0.96173176],
       [0.91728603],
       [0.94270808],
       [0.94708376],
       [0.86623681],
       [0.85803768],
       [0.87464316],
       [0.92640342],
       [0.89570847],
       [0.92166696],
       [0.95371038],
       [0.94293528],
       [0.91955065],
       [0.93513585],
       [0.95595923],
       [0.97430447],
       [0.97951847],
       [0.97033796]])

In [21]:
last_date = df.index[-1]
next_dates = []
for i in range(1, 8):  # Predict for the next 7 days to exclude weekends
    next_date = last_date + timedelta(days=i)
    if next_date.weekday() < 5:  # Check if it's a weekday (Monday to Friday)
        next_dates.append(next_date)

# Prepare input data for prediction
last_sequence = df['Price Index'][-look_back:].values.reshape(-1, 1)
input_sequence = np.array([last_sequence])

# Reshape input for LSTM prediction
input_sequence = np.reshape(input_sequence, (input_sequence.shape[0], input_sequence.shape[1], 1))

# Make predictions
predicted_prices = []
for _ in range(len(next_dates)):
    predicted_price = model.predict(input_sequence)
    predicted_prices.append(predicted_price[0, 0])
    # Shift the input sequence by one day for the next prediction
    input_sequence = np.roll(input_sequence, -1, axis=1)
    input_sequence[0, -1, 0] = predicted_price[0, 0]

# Inverse transform the predicted prices
predicted_prices = np.array(predicted_prices).reshape(-1, 1)
predicted_prices = scaler.inverse_transform(predicted_prices)

print("Predicted Price Index for the next 5 business days (excluding weekends):")
for date, price in zip(next_dates[:5], predicted_prices[:5]):
    print(date.strftime('%Y-%m-%d'), ":", price[0])


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 342ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
Predicted Price Index for the next 5 business days (excluding weekends):
2024-05-10 : 15831.54
2024-05-13 : 15801.745
2024-05-14 : 15704.02
2024-05-15 : 15578.095
2024-05-16 : 15449.79
